In [1]:
import os
os.chdir("../")

In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import torch
from torch.utils.data import Dataset, DataLoader
from utils.dataset import *
dataset = torch.load('data/dataset_bj.pt')

In [19]:
x = []
y = []
for i in range(0, len(dataset)):
    aq, meo, meo_pred, yy = dataset[i]
    aq = aq.reshape(-1)
    meo = meo.reshape(-1)
    meo_pred = meo.reshape(-1)
    yy = yy.reshape(-1)
    xx = np.concatenate([aq]) #, meo, meo_pred])
    xx[np.isnan(xx)] = 0
    yy[np.isnan(yy)] = 0
    x.append(xx)
    y.append(yy)
x = np.vstack(x)
y = np.vstack(y)
print(x.shape, y.shape)

(485, 5040) (485, 2520)


In [24]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor()

train_x = x[0: 360,]
train_y = y[0: 360,]
test_x = x[360:,]
test_y = y[360:,]
print(train_x.shape)
regr.fit(train_x, train_y)

(360, 5040)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [30]:
from utils.eval import SMAPE
result = regr.predict(test_x)
loss = []
for r, g in zip(result, test_y):
    r = r.reshape(-1, 3)
    g = g.reshape(-1, 3)
    loss.append(SMAPE(r * dataset.aq_std + dataset.aq_mean, g * dataset.aq_std + dataset.aq_mean))
np.mean(loss)

0.5904397010515304